In [ ]:
import pandas as pd
import numpy as np
np.random.seed(1337)
from keras import Sequential
from keras.utils import Sequence
from keras.layers import LSTM, Dense, Masking, GRU
import numpy as np
import keras
from keras.utils import np_utils
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Embedding, Dense, Input, concatenate, Layer, Lambda, Dropout, Activation
import datetime
from datetime import datetime
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback, TensorBoard
import tensorflow as tf
import tensorflow_hub as hub
from keras import initializers as initializers, regularizers, constraints
from keras import backend as K
import numpy as np
from numpy import load
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount("/content/Drive")

In [ ]:
dataset = pd.read_csv('/content/Drive/My Drive/LNLP/dataset.csv') # path to multi dataset
x_dev0 = load("/content/Drive/My Drive/LNLP/Hierarchical/XLNet_full_concat/XLNet_dev.npy", allow_pickle= True) # load the transformer output embeddinds

In [ ]:
x_train0_1 = load("/content/Drive/My Drive/LNLP/Hierarchical/XLNet_full_concat/XLNet_train_1.npy", allow_pickle= True)
x_train0_2 = load("/content/Drive/My Drive/LNLP/Hierarchical/XLNet_full_concat/XLNet_train_2.npy", allow_pickle= True)
x_train0_3 = load("/content/Drive/My Drive/LNLP/Hierarchical/XLNet_full_concat/XLNet_train_3.npy", allow_pickle= True)

In [ ]:
x_train0_A = np.concatenate((x_train0_1, x_train0_2))
x_train0 = np.concatenate((x_train0_A, x_train0_3))

In [ ]:
dev = dataset.loc[dataset['split'] == 'dev']
train = dataset.loc[dataset['split'] == 'train']

In [ ]:
y_train0 = []
for i in range(train.shape[0]):
  y_train0.append(train.loc[i,'label'])

y_dev0 = []
for i in range(dev.shape[0]):
  y_dev0.append(dev.loc[i+32305,'label'])

In [ ]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [ ]:
class AttentionLayer(Layer):
    """
    Hierarchial Attention Layer as described by Hierarchical Attention Networks for Document Classification(2016)
    - Yang et. al.
    Source: https://www.cs.cmu.edu/~hovy/papers/16HLT-hierarchical-attention-networks.pdf
    Theano backend
    """
    def __init__(self,attention_dim=200,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        print("attention_dim", self.attention_dim)
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        # print("shape of Weight matrix W",self.W.get_shape())
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        # print("shape of bias vector b",self.b.get_shape())
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        # print("shape of context vector u",self.u.get_shape())
        self.trainable_weights = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def _get_attention_weights(self, X):
        
        u_tw = K.tanh(K.dot(X, self.W))
        tw_stimulus = K.dot(u_tw, self.u)

        # Remove the last axis an apply softmax to the stimulus to
        # get a probability.
        tw_stimulus = K.squeeze(tw_stimulus, -1)
        tw_stimulus = K.exp(tw_stimulus)
        
        # if mask is not None:
        #     tw_stimulus *= K.cast(mask, K.floatx())

        tw_stimulus /= K.cast(K.sum(tw_stimulus, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        att_weights = K.expand_dims(tw_stimulus)
        # tw_stimulus = K.reshape(tw_stimulus, (-1, tw_stimulus.shape[1]))
        # att_weights = K.softmax(tw_stimulus)

        return att_weights

    def call(self, hit, mask=None):
        att_weights = self._get_attention_weights(hit)
        print("Shape of att_weights", att_weights.get_shape()) #(None, None, 1)
        print("Shape of hit", hit.get_shape()) #(None, None, 200)
        # Reshape the attention weights to match the dimensions of X
        # att_weights = K.reshape(att_weights, (None, None, 1))
        # att_weights = K.repeat_elements(att_weights, hit.shape[-1], -1)

        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        # print("shape of uit",uit.get_shape())
        uit = K.tanh(uit)
        # print("shape of uit",uit.get_shape())
        
        ait = K.dot(uit, self.u)
        # print("shape of ait",ait.get_shape(),ait)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        # print("shape of hit and ait",hit.get_shape(),ait.get_shape())
        # print(hit,ait)
        # print("att_weights", (ait))
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]
    #def predict_sentence_attention(self,X):


In [ ]:
from keras import layers
from tensorflow import keras
# model.add(AttentionLayer())
text_input = Input(shape=(None,3072,), dtype='float32', name='text') #1 sent_input
print("text input",text_input)
l_mask = layers.Masking(mask_value=-99.)(text_input) #2 sent_encoder
print("l_mask",l_mask)
# Which we encoded in a single vector via a LSTM
encoded_text = layers.Bidirectional(GRU(200,return_sequences=True))(l_mask) #3 sent_gru
encoded_text1 = layers.Bidirectional(GRU(200,return_sequences=True))(encoded_text) #4
encoded_text2 = layers.Bidirectional(GRU(200,return_sequences=True))(encoded_text1)
# out_dense = layers.Dense(200, activation='relu')(encoded_text1) #5 sent_dense
sent_att,sent_coeffs, = AttentionLayer(400,return_coefficients=True,name='sent_attention')(encoded_text2) #6
print("sent_att",sent_att)
print("sent_coeffs",sent_coeffs)
sent_drop = Dropout(0.5,name='sent_dropout')(sent_att)
# And we add a softmax classifier on top
out1 = layers.Dense(64, activation='relu')(sent_drop) #7 preds
out = layers.Dense(1, activation='sigmoid')(out1)
model = Model(text_input,out)
model.summary()


In [ ]:
num_sequences = len(x_train0)
batch_size = 32
batches_per_epoch =  int(num_sequences/batch_size)
#assert batch_size * batches_per_epoch == num_sequences
num_features= 3072
def train_generator():
    x_list= x_train0
    y_list =  y_train0
    # Generate batches
    while True:
        for b in range(batches_per_epoch):
            longest_index = (b + 1) * batch_size - 1
            timesteps = len(max(x_train0[:(b + 1) * batch_size][-batch_size:], key=len))
            x_train = np.full((batch_size, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size,  1))
            for i in range(batch_size):
                li = b * batch_size + i
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [ ]:
num_sequences_val = len(x_dev0)
batch_size_val = 32
batches_per_epoch_val = int(num_sequences_val/batch_size_val)
#assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 3072
def val_generator():
    x_list= x_dev0
    y_list =  y_dev0
    # Generate batches
    while True:
        for b in range(batches_per_epoch_val):
            longest_index = (b + 1) * batch_size_val - 1
            timesteps = len(max(x_dev0[:(b + 1) * batch_size_val][-batch_size_val:], key=len))
            # print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
            x_train = np.full((batch_size_val, timesteps, num_features), -99.)
            y_train = np.zeros((batch_size_val,  1))
            for i in range(batch_size_val):
                li = b * batch_size_val + i
                # print("li", li)
                # print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                x_train[i, 0:len(x_list[li]), :] = x_list[li]
                y_train[i] = y_list[li]
            yield x_train, y_train

In [ ]:
from keras.callbacks import ReduceLROnPlateau
call_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.95, patience=2, verbose=2,
                                mode='auto', min_delta=0.01, cooldown=0, min_lr=0)

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit_generator(train_generator(), steps_per_epoch=batches_per_epoch, epochs=1,
                    validation_data=val_generator(), validation_steps=batches_per_epoch_val, callbacks =[call_reduce] )
                    

In [ ]:
# x_test0 = load("BERT_test.npy", allow_pickle = True)
x_test0 = load("/content/Drive/My Drive/LNLP/Hierarchical/XLNet_full_concat/XLNet_test.npy", allow_pickle= True)
test = dataset.loc[dataset['split'] == 'test']

In [ ]:
#assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 3072
def test_generator():
    x_list= x_test0
    y_list =  y_test0
    # Generate batches
    while True:
        for b in range(batches_per_epoch_test):
            if(b == batches_per_epoch_test-1):
              # longest_index = num_sequences_val - 1
              longest_index = num_sequences_test - 1
              timesteps = len(max(x_test0[:longest_index + 1][-batch_size_test:], key=len))
              # print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
              x_train = np.full((longest_index - b*batch_size_test, timesteps, num_features), -99.)
              y_train = np.zeros((longest_index - b*batch_size_test,  1))
              for i in range(longest_index - b*batch_size_test):
                  li = b * batch_size_test + i
                  # print("li", li)
                  # print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                  x_train[i, 0:len(x_list[li]), :] = x_list[li]
                  y_train[i] = y_list[li]
            else:
                longest_index = (b + 1) * batch_size_test - 1
                timesteps = len(max(x_test0[:(b + 1) * batch_size_test][-batch_size_test:], key=len))
                # print(len(df_train['emb'].to_list()[:b+batch_size][-7:]))
                x_train = np.full((batch_size_test, timesteps, num_features), -99.)
                y_train = np.zeros((batch_size_test,  1))
                for i in range(batch_size_test):
                    li = b * batch_size_test + i
                    # print("li", li)
                    # print(x_train[i, 0:len(x_list[li]), :].shape, len(x_list[li]))
                    x_train[i, 0:len(x_list[li]), :] = x_list[li]
                    y_train[i] = y_list[li]
            yield x_train, y_train

In [ ]:
y_test0 = []
for i in range(test.shape[0]):
  y_test0.append(test.loc[i+33299,'label'])

num_sequences_test = len(x_test0)
batch_size_test = 32
if(num_sequences_test%batch_size_test == 0):
  batches_per_epoch_test = int(num_sequences_test/batch_size_test)
else:
  batches_per_epoch_test = int(num_sequences_test/batch_size_test) + 1
#assert batch_size_val * batches_per_epoch_val == num_sequences_val
num_features= 3072
model.evaluate_generator(test_generator(), steps= batches_per_epoch_test)

In [ ]:
preds = model.predict_generator(test_generator(), steps= batches_per_epoch_test)

In [ ]:
y_pred = preds > 0.5
ct=0
for i in range(len(y_pred)):
  if(y_pred[i] == y_test0[i]):
    ct+=1

print(ct/len(y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

def metrics_calculator(preds, test_labels):
    cm = confusion_matrix(test_labels, preds)
    TP = []
    FP = []
    FN = []
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[i][j]

        FN.append(summ)
    for i in range(0,2):
        summ = 0
        for j in range(0,2):
            if(i!=j):
                summ=summ+cm[j][i]

        FP.append(summ)
    for i in range(0,2):
        TP.append(cm[i][i])
    precision = []
    recall = []
    for i in range(0,2):
        precision.append(TP[i]/(TP[i] + FP[i]))
        recall.append(TP[i]/(TP[i] + FN[i]))

    macro_precision = sum(precision)/2
    macro_recall = sum(recall)/2
    micro_precision = sum(TP)/(sum(TP) + sum(FP))
    micro_recall = sum(TP)/(sum(TP) + sum(FN))
    micro_f1 = (2*micro_precision*micro_recall)/(micro_precision + micro_recall)
    macro_f1 = (2*macro_precision*macro_recall)/(macro_precision + macro_recall)
    return macro_precision, macro_recall, macro_f1, micro_precision, micro_recall, micro_f1


In [ ]:
print(metrics_calculator(y_pred, y_test0[:-1]))

In [ ]:
preds = model.predict_generator(val_generator(), steps= batches_per_epoch_val)

In [ ]:
y_pred_dev = preds > 0.5

In [ ]:
ct=0
for i in range(len(y_pred_dev)):
  if(y_pred_dev[i] == y_dev0[i]):
    ct+=1

print(ct/len(y_pred_dev))

In [ ]:
print(metrics_calculator(y_pred_dev, y_dev0[:-2]))

In [ ]:
import os
# if not os.path.exists(os.path.join('..', 'h5.models')):
#   os.mkdir(os.path.join('/content/Drive/My Drive/shared_by_vijit/LNLP/HAN/RoBERTa/', 'Roberta_gru_3_layer_attention'))
model.save(os.path.join('/content/Drive/My Drive/LNLP/Hierarchical/XLNet_full_concat/', '', 'XGA_concat_epoch1_3.h5'))